In [2]:
import torch
import numpy as np

In [160]:
import cvxpy as cp


x = cp.Variable()
y = cp.Variable()
gamma = cp.Parameter(nonneg=True)
problem = cp.Problem(cp.Minimize(gamma * y), [y == gamma * x])
print("Is DPP? ", problem.is_dcp(dpp=True))
print("Is DCP? ", problem.is_dcp(dpp=False))

Is DPP?  True
Is DCP?  True


In [166]:
a = torch.tensor([[1],[2]])
b = torch.tensor([[0]])
torch.cat((a,b),0).T
print(b - 1.0)

tensor([[-1.]])


In [155]:
class dynamics(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        # x_{t+1} = f(x_t) + g(x_t)u_t
        x = input[0:2,0]
        u = input[2:4,0]
        fx = x
        gx = torch.tensor([[1,0],[0,1]],dtype=torch.float)
        return fx + torch.matmul(gx, u)
    
    @staticmethod
    def backward(ctx, grad_output):
        print("hello")
        input, = ctx.saved_tensors
#         grad_input = grad_output.clone()
        input, = ctx.saved_tensors
        x = input[0:2].detach().numpy()
        u = input[2:4].detach().numpy()
        gx = np.array([[1,0],[0,1]])
        gxu = np.array([[1,0],[0,1]]) @ u
        df_dx =  np.array([[1,0],[0,1]])
        dgxu_dx = np.array([[0,0],[0,0]])
#         print(dg_dx)
        print(u)
        gradient = torch.tensor(np.append(df_dx+dgxu_dx, gx, axis=1),dtype=torch.float)
#         print("gradient",gradient.size())
#         print("grad_output",grad_output.size())
#         print("grad_output",grad_output)
        return torch.reshape(torch.matmul(torch.reshape(grad_output,(1,2)),gradient),(4,1))
        

In [159]:
dtype = torch.float
device = torch.device("cpu")

dynam = dynamics.apply
x = np.array([1,1]).reshape(-1,1)
u = np.array([5,2]).reshape(-1,1)
input = torch.tensor(np.append(x,u,axis=0),dtype=dtype, requires_grad=True)
y_pred = dynam(input)
print(y_pred)
# loss = y_pred.sum()
# loss.backward()
# print(input.grad)
# print("xx",np.shape(x))

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

x = cp.Variable(2)
y = cp.Parameter(2)
objective = cp.Minimize(0.5*cp.norm(x-y/2))
const = [x[0]<=10]
problem = cp.Problem(objective, const)
assert problem.is_dpp()
# print(problem.parameters())

cvxpylayer = CvxpyLayer(problem, parameters=[y], variables=[x])
# this is just y_pred, a_tch = torch.tensor(a.value, requires_grad=True, dtype=torch.float)

# solve the problem
solution, = cvxpylayer(y_pred)

print(solution.detach().numpy())
loss = solution.sum()
print("loss",loss)
loss.backward()
print(input.grad)

tensor([6., 3.], grad_fn=<dynamicsBackward>)
[3.  1.5]
loss tensor(4.5000, grad_fn=<SumBackward0>)
hello
[[5.]
 [2.]]
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]])


In [143]:
import time
import cvxpy as cp
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import torch
x = cp.Variable((2,1))
a = cp.Parameter((2,1),value=np.array([[.5],[.5]]))
c = cp.Parameter((2,1),value=np.array([[-2],[-2]]))
const = [x<=c]
s = time.time()
objective = cp.Minimize(0.5*cp.norm(x-a))
problem = cp.Problem(objective, const)
assert problem.is_dpp()
print(problem.parameters())

cvxpylayer = CvxpyLayer(problem, parameters=[a, c], variables=[x])
a_tch = torch.tensor(a.value, requires_grad=True, dtype=torch.float)
c_tch = torch.tensor(c.value, requires_grad=True, dtype=torch.float)

# solve the problem
solution, = cvxpylayer(a_tch, c_tch)
print(solution.detach().numpy())
print(solution)
# print(x.value)
e1 = torch.tensor(np.array([1.0,0]), dtype=torch.float)
e2 = torch.tensor(np.array([0.0,1.0]), dtype=torch.float)

# # compute the gradient of the sum of the solution with respect to a, c
# solution.sum().backward()
u1 = torch.matmul(e1,solution)
u2 = torch.matmul(e2,solution)
print("u1",u1)

u1.backward(retain_graph=True)
print("a grad ", a_tch.grad.numpy())
print("c grad ", c_tch.grad.numpy())

c_prev = np.copy(c_tch.grad.numpy())


print("u2",u2)
u2.backward(retain_graph=False)
print(time.time()-s)
print("a grad ", a_tch.grad.numpy().reshape(1,-1))
print("c grad ", c_tch.grad.numpy())
print("c_prev",c_prev)
print("c grad diff", c_tch.grad.numpy()-c_prev)
# problem.solve()
# print(problem.status)
# print(x.value)

[Parameter((2, 1)), Parameter((2, 1))]
[[-2.]
 [-2.]]
tensor([[-2.],
        [-2.]], grad_fn=<_CvxpyLayerFnFnBackward>)
u1 tensor([-2.], grad_fn=<SqueezeBackward3>)
a grad  [[-1.7689322e-13]
 [-1.7699470e-13]]
c grad  [[1.0000000e+00]
 [1.5122192e-13]]
u2 tensor([-2.], grad_fn=<SqueezeBackward3>)
0.027474641799926758
a grad  [[-3.5071859e-13 -3.5079405e-13]]
c grad  [[1.]
 [1.]]
c_prev [[1.0000000e+00]
 [1.5122192e-13]]
c grad diff [[0.]
 [1.]]


In [14]:
print(solution)

tensor([[-2.],
        [-2.]], grad_fn=<_CvxpyLayerFnFnBackward>)


In [15]:
u = torch.tensor(np.array([[1],[1]]),dtype=dtype, requires_grad=True)
input = torch.cat((solution,u),axis=0)

In [16]:
y_pred = dynam(input)
print(y_pred)
loss = y_pred.sum()
loss.backward()
print(c_tch.grad)

tensor([-1., -1.], grad_fn=<dynamicsBackward>)
hello
[[1.]
 [1.]]
gradient torch.Size([2, 4])
grad_output torch.Size([2])
tensor([[1.],
        [1.]])


In [124]:
a = np.array([0])
b = a

In [125]:
b[0]=3

In [126]:
a

array([3])

In [127]:
b = np.copy(a)

In [128]:
b[0]=5

In [129]:
b

array([5])

In [107]:
class robot:
    
    def fx(self,x):
        return x
    
    def gx(self,x):
        return torch.tensor(np.array([[1,0],[0,1]]),dtype=torch.float);
    
    def df_dx(self,x, type = 'tensor'):
        if type == 'numpy':
            return np.array(np.array([[1,0],[0,1]]))
        if type == 'tensor':
            return torch.tensor([[1,0],[0,1]],dtype=torch.float)
        
    def dgxu_dx(self,x, type='tensor'):
        if type == 'numpy':
            return np.array([[0,0],[0,0]])
        if type == 'tensor':
            return torch.tensor([[0,0],[0,0]],dtype=torch.float)
        

In [139]:
def fx_(x):
    return x

def gx_(x):
    return torch.tensor(np.array([[1,0],[0,1]]),dtype=torch.float);

def df_dx_(x, type = 'tensor'):
    if type == 'numpy':
        return np.array(np.array([[1,0],[0,1]]))
    if type == 'tensor':
        return torch.tensor([[1,0],[0,1]],dtype=torch.float)
    
def dgxu_dx_(x, type='tensor'):
    if type == 'numpy':
        return np.array([[0,0],[0,0]])
    if type == 'tensor':
        return torch.tensor([[0,0],[0,0]],dtype=torch.float)

In [134]:
class torch_dynamics(torch.autograd.Function):
    
    def __init__(self, robot):
        self.robot = robot

    @staticmethod
    def forward(ctx, input_x, input_u, robot):
#         ctx.save_for_backward(input)
        # x_{t+1} = f(x_t) + g(x_t)u_t
        x = input_x
        u = input_u
        fx = robot.fx(x)  #tensor
        gx = robot.gx(x)  #tensor
        
#         gxu = torch.tensor(robot.gx(x).detach().numpy() @ u.detach().numpy()),dtype=torch.float)
        gxu = torch.matmul(gx, u)
        df_dx =  robot.df_dx(x) # numpy
        dgxu_dx = robot.dgxu_dx(x)  # numpy
        
        print(gx, gxu, df_dx, dgxu_dx)
        
        ctx.save_for_backward(x,u, gx, gxu, df_dx, dgxu_dx)

        return fx + torch.matmul(gx, u)

    @staticmethod
    def backward(ctx, grad_output):

        input_x, input_u, gx, gxu, df_dx, dgxu_dx, = ctx.saved_tensors
        x = input_x.detach().numpy()
        u = input_u.detach().numpy()
        
        print(grad_output)
        
        gradient = torch.tensor(np.append(df_dx+dgxu_dx, gx, axis=1),dtype=torch.float)
        print("gradient",gradient.size())
        print("hello")
        print("grad_output",grad_output.size())
        print("grad_output",grad_output)
        print( torch.reshape(torch.matmul(torch.reshape(grad_output,(1,2)),gradient),(4,1)) )
        return torch.reshape(torch.matmul(torch.reshape(grad_output,(1,2)),gradient),(4,1))

In [136]:
dtype = torch.float
device = torch.device("cpu")

rob = robot()

dynam = torch_dynamics.apply
x = torch.tensor(np.array([1,1]).reshape(-1,1),dtype=dtype, requires_grad=True)
u = torch.tensor(np.array([3,2]).reshape(-1,1),dtype=dtype, requires_grad=True)
# input = torch.tensor(np.append(x,u,axis=0),dtype=dtype, requires_grad=True)
y_pred = dynam(x,u, rob)
print(y_pred)
loss = y_pred.sum()
loss.backward()
print(input.grad)

xx = x.detach().numpy()
print(np.shape(xx))

tensor([[1., 0.],
        [0., 1.]]) tensor([[3.],
        [2.]]) tensor([[1., 0.],
        [0., 1.]]) tensor([[0., 0.],
        [0., 0.]])
tensor([[4.],
        [3.]], grad_fn=<torch_dynamicsBackward>)
tensor([[1.],
        [1.]])
gradient torch.Size([2, 4])
hello
grad_output torch.Size([2, 1])
grad_output tensor([[1.],
        [1.]])
tensor([[1.],
        [1.],
        [1.],
        [1.]])


RuntimeError: function torch_dynamicsBackward returned an incorrect number of gradients (expected 3, got 1)